# 1. Generate fa files for deeplift

In [ ]:
import numpy as np
import pybedtools
import pandas as pd
from Bio import SeqIO
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
starr_peak = pd.read_csv('../data/starr_sig_A549_499bp/data/A549_training_499bp_7_windows_100_stride_random_region_value_label_10_labels_sorted.bed', sep='\t', header=None)

In [ ]:
starr_peak[4].value_counts()

1    510651
0    391830
2     94162
3     18261
4      6242
5      2816
6      1405
9      1204
7       862
8       520
Name: 4, dtype: int64

In [ ]:
starr_peak = starr_peak.loc[(starr_peak[4] >= 4), :]
starr_peak

,0,1,2,3,4
144,chr1,790202,790701,+,4
201,chr1,1006493,1006992,+,6
284,chr1,1264807,1265306,+,6
572,chr1,2097551,2098050,+,4
573,chr1,2097651,2098150,+,7
...,...,...,...,...,...
1021816,chrY,21151299,21151798,+,8
1021817,chrY,21151648,21152147,+,9
1021818,chrY,21151997,21152496,+,6
1021819,chrY,21152097,21152596,+,4


In [ ]:
seq = pybedtools.BedTool.from_dataframe(starr_peak[[0, 1, 2]]).sequence(fi='../genomes/hg38.fa')
seq.save_seqs(f'../data/output_data/starr_sig_valueLabel_499bp.fa')

<BedTool(/tmp/pybedtools.ka56ty9l.tmp)>

In [ ]:
starr_peak = pd.read_csv('../data/starr_sig_A549_499bp/data/A549_replicate1_2_499bp_7_windows_100_stride_random_region_value_label_10_labels_sorted.bed', sep='\t', header=None)

In [ ]:
starr_peak[4].value_counts()

1    473687
0    395596
2     89314
3     18344
4      6278
5      2822
6      1428
9      1231
7       862
8       514
Name: 4, dtype: int64

In [ ]:
starr_peak = starr_peak.loc[(starr_peak[4] >= 4), :]
starr_peak

,0,1,2,3,4
201,chr1,1006437,1006936,+,6
223,chr1,1053783,1054282,+,4
284,chr1,1264811,1265310,+,6
523,chr1,2097551,2098050,+,4
524,chr1,2097651,2098150,+,7
...,...,...,...,...,...
983953,chrY,21151301,21151800,+,7
983954,chrY,21151650,21152149,+,9
983955,chrY,21151999,21152498,+,6
983956,chrY,21152099,21152598,+,4


In [ ]:
seq = pybedtools.BedTool.from_dataframe(starr_peak[[0, 1, 2]]).sequence(fi='../genomes/hg38.fa')
seq.save_seqs(f'../data/output_data/starr_sig_valueLabel_499bp_rep1rep2.fa')

<BedTool(/tmp/pybedtools._foy5ndc.tmp)>

# 2. Convert trained model to deeplift required format

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import gmean
import pprint
import pybedtools
from collections import OrderedDict
from torch.autograd import Variable
from pytorch2keras import pytorch_to_keras
from Bio import SeqIO
from tensorflow import keras
import tensorflow as tf
%matplotlib inline

## 2.1 499bp

### 2.1.1 Read in trained pytorch model without reshape

In [ ]:
class LambdaBase(nn.Sequential):
    def __init__(self, fn, *args):
        super(LambdaBase, self).__init__(*args)
        self.lambda_func = fn

    def forward_prepare(self, input):
        output = []
        for module in self._modules.values():
            output.append(module(input))
        return output if output else input

    
class Lambda(LambdaBase):
    def forward(self, input):
        return self.lambda_func(self.forward_prepare(input))


class Beluga(nn.Module):
    def __init__(self, sequence_length, n_targets):
        super(Beluga, self).__init__()
        conv_kernel_size = 8
        pool_kernel_size = 4
        self.model = nn.Sequential(
            #Lambda(lambda x: x.reshape(-1,4,1,sequence_length)), #added, and changed from view to reshape
            nn.Conv2d(4,320,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Conv2d(320,320,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d((1, pool_kernel_size),(1, pool_kernel_size)),
            nn.Conv2d(320,480,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Conv2d(480,480,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d((1, pool_kernel_size),(1, pool_kernel_size)),
            nn.Conv2d(480,640,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Conv2d(640,640,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Dropout(0.5),
            Flatten(),
            nn.Linear(7680,2003), #500: 7680, 1000:28160, 2000:67840
            nn.ReLU(),
            nn.Linear(2003,n_targets),
            nn.Sigmoid()
            )
#                 Lambda(lambda x: x.reshape(x.size(0),-1)),
#                 nn.Sequential(Lambda(lambda x: x.reshape(1,-1) if 1==len(x.size()) else x ),nn.Linear(67840,2003)),
#                 nn.ReLU(),
#                 nn.Sequential(Lambda(lambda x: x.reshape(1,-1) if 1==len(x.size()) else x ),nn.Linear(2003,n_targets)),

    def forward(self, x):
        return self.model(x)
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
model = Beluga(sequence_length=499, n_targets=1)

In [ ]:
torch.__version__

'1.8.1+cu102'

In [ ]:
all_starr_state_dict = torch.load('../data/starr_sig_A549_499bp/training_outputs/transfer_trained_rep1rep2/best_model.pth.tar')['state_dict']
all_starr_state_dict_new = OrderedDict()

In [ ]:
for model_key, trained_model_key in zip(model.state_dict().keys(), all_starr_state_dict.keys()):
    all_starr_state_dict_new[model_key] = all_starr_state_dict[trained_model_key]

In [ ]:
model.state_dict()

In [ ]:
model.load_state_dict(all_starr_state_dict_new)

<All keys matched successfully>

In [ ]:
model.state_dict()

In [ ]:
del all_starr_state_dict, all_starr_state_dict_new

### 2.1.2 Convert to keras model 

In [ ]:
input_np = np.random.uniform(0, 1, (1, 4, 1, 499))
input_var = Variable(torch.FloatTensor(input_np))

In [ ]:
k_model = pytorch_to_keras(model, input_var, verbose=True, change_ordering=True)

In [ ]:
k_model.summary()

In [ ]:
k_model.save('../models/converted_models/keras_model.h5')

### 2.1.3 Convert to keras sequential model with conv1d

In [ ]:
k_model_1d = keras.models.Sequential()
k_model_1d.add(keras.layers.Conv1D(filters=320, kernel_size=8, input_shape=(499,4))) #17
k_model_1d.add(keras.layers.Activation("relu")) #18
k_model_1d.add(keras.layers.Conv1D(filters=320, kernel_size=8)) #19
k_model_1d.add(keras.layers.Activation("relu")) #20
k_model_1d.add(keras.layers.MaxPooling1D(pool_size=4))#21
k_model_1d.add(keras.layers.Conv1D(filters=480, kernel_size=8)) #22
k_model_1d.add(keras.layers.Activation("relu")) #23
k_model_1d.add(keras.layers.Conv1D(filters=480, kernel_size=8)) #24
k_model_1d.add(keras.layers.Activation("relu")) #25
k_model_1d.add(keras.layers.MaxPooling1D(pool_size=4))#26
k_model_1d.add(keras.layers.Conv1D(filters=640, kernel_size=8)) #27
k_model_1d.add(keras.layers.Activation("relu")) #28
k_model_1d.add(keras.layers.Conv1D(filters=640, kernel_size=8)) #29
k_model_1d.add(keras.layers.Activation("relu")) #30
# k_model_1d.add(keras.layers.Permute((2, 1))) #31_CHW
k_model_1d.add(keras.layers.Flatten(data_format='channels_first')) #31
k_model_1d.add(keras.layers.Dense(units=2003)) #32
k_model_1d.add(keras.layers.Activation("relu")) #33
k_model_1d.add(keras.layers.Dense(units=1)) #34
k_model_1d.add(keras.layers.Activation("softmax")) #sigmoid
k_model_1d.build()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
k_model = keras.models.load_model('../models/converted_models/keras_model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2021-12-04 19:49:43.124046: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-12-04 19:49:43.165921: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2194965000 Hz
2021-12-04 19:49:43.173932: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55828bf71a60 executing computations on platform Host. Devices:
2021-12-04 19:49:43.173994: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-12-04 19:49:43.175915: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2021-12-04 19:49:43.218216: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: TITAN V major: 7 minor: 0 memoryClockRate(GHz): 1.455
pciBusID: 0000:02:00.0
2021-12-04 19:49:43.218273: I tensorflow/stream_executor/platfor

In [ ]:
trans_layer_index_k_model_1d = [0, 2, 5, 7, 10, 12, 15, 17]
trans_layer_index_k_model = [1, 3, 6, 8, 11, 13, 17, 19]

In [ ]:
for index_k_model_1d, index_k_model in zip(trans_layer_index_k_model_1d, trans_layer_index_k_model):
    try:
        k_model_1d.layers[index_k_model_1d].set_weights(
            [np.squeeze(k_model.layers[index_k_model].get_weights()[0]), 
             k_model.layers[index_k_model].get_weights()[1]])
    except ValueError:
        k_model_1d.layers[index_k_model_1d].set_weights(
            [k_model.layers[index_k_model].get_weights()[0], 
             k_model.layers[index_k_model].get_weights()[1]])

In [ ]:
k_model_1d.save('../models/converted_models/starr_sig_A549_499bp_rep1rep2.h5')

## 2.2 499bp regression 

### 2.2.1 Read in trained pytorch model without reshape

In [ ]:
class LambdaBase(nn.Sequential):
    def __init__(self, fn, *args):
        super(LambdaBase, self).__init__(*args)
        self.lambda_func = fn

    def forward_prepare(self, input):
        output = []
        for module in self._modules.values():
            output.append(module(input))
        return output if output else input

    
class Lambda(LambdaBase):
    def forward(self, input):
        return self.lambda_func(self.forward_prepare(input))


class Beluga(nn.Module):
    def __init__(self, sequence_length, n_targets):
        super(Beluga, self).__init__()
        conv_kernel_size = 8
        pool_kernel_size = 4
        self.model = nn.Sequential(
            #Lambda(lambda x: x.reshape(-1,4,1,sequence_length)), #added, and changed from view to reshape
            nn.Conv2d(4,320,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Conv2d(320,320,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d((1, pool_kernel_size),(1, pool_kernel_size)),
            nn.Conv2d(320,480,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Conv2d(480,480,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d((1, pool_kernel_size),(1, pool_kernel_size)),
            nn.Conv2d(480,640,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Conv2d(640,640,(1, conv_kernel_size)),
            nn.ReLU(),
            nn.Dropout(0.5),
            Flatten(),
            nn.Linear(7680,2003), #500: 7680, 1000:28160, 2000:67840
            nn.ReLU(),
            nn.Linear(2003,n_targets),
            #nn.Softmax(),#nn.Sigmoid()
            )
#                 Lambda(lambda x: x.reshape(x.size(0),-1)),
#                 nn.Sequential(Lambda(lambda x: x.reshape(1,-1) if 1==len(x.size()) else x ),nn.Linear(67840,2003)),
#                 nn.ReLU(),
#                 nn.Sequential(Lambda(lambda x: x.reshape(1,-1) if 1==len(x.size()) else x ),nn.Linear(2003,n_targets)),

    def forward(self, x):
        return self.model(x)
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
model = Beluga(sequence_length=499, n_targets=1)

In [ ]:
torch.__version__

'1.8.1+cu102'

In [ ]:
all_starr_state_dict = torch.load('../data/starr_sig_A549_499bp_regression/training_outputs/transfer_trained_rep1rep2/best_model.pth.tar')['state_dict']
all_starr_state_dict_new = OrderedDict()

In [ ]:
for model_key, trained_model_key in zip(model.state_dict().keys(), all_starr_state_dict.keys()):
    all_starr_state_dict_new[model_key] = all_starr_state_dict[trained_model_key]

In [ ]:
model.state_dict()

OrderedDict([('model.0.weight',
              tensor([[[[-0.0129,  0.1245, -0.0186,  ...,  0.0815,  0.1174, -0.1663]],
              
                       [[ 0.1011, -0.1691,  0.1431,  ...,  0.0809,  0.0417, -0.0384]],
              
                       [[ 0.0777,  0.0072, -0.1290,  ...,  0.0088, -0.1462,  0.0324]],
              
                       [[ 0.1284, -0.1667, -0.0276,  ...,  0.1104, -0.0075,  0.0639]]],
              
              
                      [[[ 0.0854,  0.1462,  0.0444,  ..., -0.1547,  0.0120, -0.1536]],
              
                       [[-0.0359,  0.0775, -0.1079,  ...,  0.0069, -0.0884, -0.1137]],
              
                       [[ 0.1046, -0.0147, -0.0422,  ...,  0.1220,  0.0317, -0.0919]],
              
                       [[-0.0524,  0.0262,  0.1078,  ...,  0.0783, -0.1059, -0.1555]]],
              
              
                      [[[-0.1700, -0.0174, -0.1754,  ..., -0.0269, -0.0415, -0.0130]],
              
                  

In [ ]:
model.load_state_dict(all_starr_state_dict_new)

<All keys matched successfully>

In [ ]:
model.state_dict()

OrderedDict([('model.0.weight',
              tensor([[[[ 1.6613e-01,  3.1092e-01, -1.0372e-01,  ..., -1.9600e-01,
                         -7.4552e-02,  5.0334e-02]],
              
                       [[-1.1445e-01, -4.7423e-02,  1.4395e-01,  ...,  3.0431e-01,
                          1.5359e-01, -2.3749e-01]],
              
                       [[-2.5124e-01, -2.4415e-01, -2.5052e-01,  ..., -2.1614e-01,
                         -3.1528e-01,  1.5794e-01]],
              
                       [[-3.6601e-02, -1.4999e-01, -2.3390e-01,  ..., -1.0493e-01,
                          2.3972e-04, -1.4323e-01]]],
              
              
                      [[[-4.1824e-02,  9.0586e-02,  4.6133e-01,  ..., -2.4137e-01,
                          3.1201e-01, -1.8993e-01]],
              
                       [[ 1.7946e-03, -4.2285e-01, -2.6302e-01,  ...,  1.9758e-01,
                         -2.6901e-01, -2.6468e-01]],
              
                       [[-1.7176e-01,  2.3972e

In [ ]:
del all_starr_state_dict, all_starr_state_dict_new

### 2.2.2 Convert to keras model 

In [ ]:
input_np = np.random.uniform(0, 1, (1, 4, 1, 499))
input_var = Variable(torch.FloatTensor(input_np))

In [ ]:
k_model = pytorch_to_keras(model, input_var, verbose=True, change_ordering=True)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight model.0.bias with shape (320,).
DEBUG:onnx2keras:Found weight model.0.weight with shape (320, 4, 1, 8).
DEBUG:onnx2keras:Found weight model.12.bias with shape (640,).
DEBUG:onnx2keras:Found weight model.12.weight with shape (640, 480, 1, 8).
DEBUG:onnx2keras:Found weight model.14.bias with shape (640,).
DEBUG:onnx2keras:Found weight model.14.weight with shape (640, 640, 1, 8).
DEBUG:onnx2keras:Found weight model.18.bias with shape (2003,).
DEBUG:onnx2keras:Found weight model.18.weight with shape (2003, 7680).


graph(%input_0 : Float(1, 4, 1, 499),
      %model.0.weight : Float(320, 4, 1, 8),
      %model.0.bias : Float(320),
      %model.2.weight : Float(320, 320, 1, 8),
      %model.2.bias : Float(320),
      %model.6.weight : Float(480, 320, 1, 8),
      %model.6.bias : Float(480),
      %model.8.weight : Float(480, 480, 1, 8),
      %model.8.bias : Float(480),
      %model.12.weight : Float(640, 480, 1, 8),
      %model.12.bias : Float(640),
      %model.14.weight : Float(640, 640, 1, 8),
      %model.14.bias : Float(640),
      %model.18.weight : Float(2003, 7680),
      %model.18.bias : Float(2003),
      %model.20.weight : Float(1, 2003),
      %model.20.bias : Float(1)):
  %17 : Float(1, 320, 1, 492) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 8], pads=[0, 0, 0, 0], strides=[1, 1]](%input_0, %model.0.weight, %model.0.bias) # /data/software/Anaconda3/4.4.0-foss-2017a/envs/deeplift_modisco/lib/python3.7/site-packages/torch/nn/modules/conv.py:342:0
  %18 : Float(1, 320, 1, 4

DEBUG:onnx2keras:Found weight model.2.bias with shape (320,).
DEBUG:onnx2keras:Found weight model.2.weight with shape (320, 320, 1, 8).
DEBUG:onnx2keras:Found weight model.20.bias with shape (1,).
DEBUG:onnx2keras:Found weight model.20.weight with shape (1, 2003).
DEBUG:onnx2keras:Found weight model.6.bias with shape (480,).
DEBUG:onnx2keras:Found weight model.6.weight with shape (480, 320, 1, 8).
DEBUG:onnx2keras:Found weight model.8.bias with shape (480,).
DEBUG:onnx2keras:Found weight model.8.weight with shape (480, 480, 1, 8).
DEBUG:onnx2keras:Found input input_0 with shape [4, 1, 499]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 17
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 8], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': True, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:on

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
k_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1, 499, 4)]       0         
_________________________________________________________________
17 (Conv2D)                  (None, 1, 492, 320)       10560     
_________________________________________________________________
18 (Activation)              (None, 1, 492, 320)       0         
_________________________________________________________________
19 (Conv2D)                  (None, 1, 485, 320)       819520    
_________________________________________________________________
20 (Activation)              (None, 1, 485, 320)       0         
_________________________________________________________________
21 (MaxPooling2D)            (None, 1, 121, 320)       0         
_________________________________________________________________
22 (Conv2D)                  (None, 1, 114, 480)       122928

In [ ]:
k_model.save('./converted_models/keras_model.h5')

### 2.2.3 Convert to keras sequential model with conv1d

In [ ]:
k_model_1d = keras.models.Sequential()
k_model_1d.add(keras.layers.Conv1D(filters=320, kernel_size=8, input_shape=(499,4))) #17
k_model_1d.add(keras.layers.Activation("relu")) #18
k_model_1d.add(keras.layers.Conv1D(filters=320, kernel_size=8)) #19
k_model_1d.add(keras.layers.Activation("relu")) #20
k_model_1d.add(keras.layers.MaxPooling1D(pool_size=4))#21
k_model_1d.add(keras.layers.Conv1D(filters=480, kernel_size=8)) #22
k_model_1d.add(keras.layers.Activation("relu")) #23
k_model_1d.add(keras.layers.Conv1D(filters=480, kernel_size=8)) #24
k_model_1d.add(keras.layers.Activation("relu")) #25
k_model_1d.add(keras.layers.MaxPooling1D(pool_size=4))#26
k_model_1d.add(keras.layers.Conv1D(filters=640, kernel_size=8)) #27
k_model_1d.add(keras.layers.Activation("relu")) #28
k_model_1d.add(keras.layers.Conv1D(filters=640, kernel_size=8)) #29
k_model_1d.add(keras.layers.Activation("relu")) #30
# k_model_1d.add(keras.layers.Permute((2, 1))) #31_CHW
k_model_1d.add(keras.layers.Flatten(data_format='channels_first')) #31
k_model_1d.add(keras.layers.Dense(units=2003)) #32
k_model_1d.add(keras.layers.Activation("relu")) #33
k_model_1d.add(keras.layers.Dense(units=1)) #34
# k_model_1d.add(keras.layers.Activation("softmax")) #sigmoid
k_model_1d.build()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
k_model = keras.models.load_model('../models/converted_models/keras_model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2021-12-04 19:49:43.124046: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-12-04 19:49:43.165921: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2194965000 Hz
2021-12-04 19:49:43.173932: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55828bf71a60 executing computations on platform Host. Devices:
2021-12-04 19:49:43.173994: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-12-04 19:49:43.175915: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2021-12-04 19:49:43.218216: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: TITAN V major: 7 minor: 0 memoryClockRate(GHz): 1.455
pciBusID: 0000:02:00.0
2021-12-04 19:49:43.218273: I tensorflow/stream_executor/platfor

In [ ]:
trans_layer_index_k_model_1d = [0, 2, 5, 7, 10, 12, 15, 17]
trans_layer_index_k_model = [1, 3, 6, 8, 11, 13, 17, 19]

In [ ]:
for index_k_model_1d, index_k_model in zip(trans_layer_index_k_model_1d, trans_layer_index_k_model):
    try:
        k_model_1d.layers[index_k_model_1d].set_weights(
            [np.squeeze(k_model.layers[index_k_model].get_weights()[0]), 
             k_model.layers[index_k_model].get_weights()[1]])
    except ValueError:
        k_model_1d.layers[index_k_model_1d].set_weights(
            [k_model.layers[index_k_model].get_weights()[0], 
             k_model.layers[index_k_model].get_weights()[1]])

In [ ]:
k_model_1d.save('../models/converted_models/starr_sig_A549_499bp_rep1rep2_regression.h5')